In [190]:
import azureml.core
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.exceptions import ComputeTargetException
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.train.dnn import TensorFlow
# from azureml.train.hyperdrive import *
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

import os
import shutil
import urllib
import numpy as np
import matplotlib.pyplot as plt

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.16.0


In [191]:
#from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

exp = Experiment(workspace=ws, name="udacity-project")


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: AML
Azure region: westus
Subscription id: 4dc660fa-114b-4613-ae16-cb4015161f3d
Resource group: Udacity-AML


In [192]:
script_folder = './tf-mnist'
os.makedirs(script_folder, exist_ok=True)

#from azureml.core.experiment import Experiment
exp = Experiment(workspace=ws, name='udacity-project')

In [193]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name = "amlcomp"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target {}.'.format(cluster_name))
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                               max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)

print("Azure Machine Learning Compute attached")

cpu_cluster_name = "cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    cpu_cluster.wait_for_completion(show_output=True)

Found existing compute target amlcomp.
Azure Machine Learning Compute attached
Found existing cpu-cluster


In [203]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
#from azureml.train.hyperdrive import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import shutil
import os




# Specify parameter sampler
#ps = ### YOUR CODE HERE ###
#ps = RandomParameterSampling(
#    {
#        '--batch-size': choice(25, 50, 100),
#        '--first-layer-neurons': choice(10, 50, 200, 300, 500),
#        '--second-layer-neurons': choice(10, 50, 200, 500),
#        '--learning-rate': loguniform(-6, -1)
#    }
#)

sample_space = {
    'C': choice(0.01, 0.1, 1, 10, 100),
    'max_iter' : choice(50,75,100,125,150,175,200),
    'solver' : choice('liblinear','sag','lbfgs', 'saga')
}
ps = RandomParameterSampling(sample_space)


# Specify a Policy
#policy = ### YOUR CODE HERE ###
#policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)
policy = MedianStoppingPolicy(evaluation_interval=1,delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")
shutil.copy('./train.py','./training')

# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###
est = SKLearn(
    source_directory = './training', 
    compute_target = compute_target, 
    entry_script = 'train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###
    
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    max_total_runs=8, 
    max_concurrent_runs=4,
    hyperparameter_sampling=ps, 
    primary_metric_name='Accuracy', 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    policy=policy
    
)

In [204]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.widgets import RunDetails

hd_submit = exp.submit(config=hyperdrive_config)

RunDetails(hd_submit).show()

hd_submit.wait_for_completion(show_output=True)



WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_1eeb4b3b-8b35-4cf5-b451-a4e7e34ee59b
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_1eeb4b3b-8b35-4cf5-b451-a4e7e34ee59b?wsid=/subscriptions/4dc660fa-114b-4613-ae16-cb4015161f3d/resourcegroups/Udacity-AML/workspaces/AML

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-10-29T13:45:14.229213][API][INFO]Experiment created<END>\n""<START>[2020-10-29T13:45:15.014958][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-10-29T13:45:15.379629][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-10-29T13:45:16.0593646Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_submission.get_best_run_by_primary_metric()
#best_run = hd_submit.get_best_run_by_primary_metric()
best_run = hyperDrive_run.get_best_run_by_primary_metric()
print('\n')
print(best_run)



In [180]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

dataset_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data = TabularDatasetFactory.from_delimited_files(path=dataset_path)

WARNING - Connection pool is full, discarding connection: westus.api.azureml.ms
WARNING - Connection pool is full, discarding connection: westus.api.azureml.ms
WARNING - Connection pool is full, discarding connection: westus.api.azureml.ms
WARNING - Connection pool is full, discarding connection: westus.api.azureml.ms
WARNING - Connection pool is full, discarding connection: westus.api.azureml.ms
WARNING - Connection pool is full, discarding connection: westus.api.azureml.ms
WARNING - Connection pool is full, discarding connection: westus.api.azureml.ms


In [181]:
from train import clean_data

# Use the clean_data function to clean your data.
#x, y = clean_data(### YOUR DATA OBJECT HERE ###)
    
x, y = clean_data(input_data)
predict_col = pd.DataFrame(y)
predict_col.head()
final_data = pd.concat([x,predict_col],axis=1)
final_data.head()

ModuleNotFoundError: No module named 'easydict'

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(final_data,test_size=0.3)
train_data.head()

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###